In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F

question 1

In [25]:
pyspark.version

<module 'pyspark.version' from '/workspaces/LKDEZoomcamp2024/spark/spark-3.3.2-bin-hadoop3/python/pyspark/version.py'>

In [2]:
credentials_location = '/workspaces/LKDEZoomcamp2024/4_datawarehouse/onyx-descent-417702-c2fbbb866ff5-magedezoomcamp.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/workspaces/LKDEZoomcamp2024/spark/gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/04/18 07:19:55 WARN Utils: Your hostname, codespaces-d61839 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/04/18 07:19:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/18 07:19:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [5]:
df_fhv = spark.read \
    .option("InferSchema", "true") \
    .parquet('gs://airflow-lkdezoomcamp/raw/fhv_taxi/fhv_tripdata_2019-10.parquet')

In [7]:
df_fhv.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: long (nullable = true)
 |-- DOlocationID: long (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



question 2

go to bucket and look at the size
should be 6.2mb

In [8]:
df_fhv.repartition(6) \
    .write.parquet('gs://airflow-lkdezoomcamp/pq/fhv/2019_10/', mode='overwrite')

In [9]:
df_fhv_2 = df_fhv \
            .withColumn('pickup_date', F.to_date(df_fhv.pickup_datetime)) \
            .withColumn('dropoff_date', F.to_date(df_fhv.dropOff_datetime)) \
            .withColumn('trip_duration_hours', (F.col('dropOff_datetime').cast('long') - F.col('pickup_datetime').cast('long'))/3600)

question 3

In [11]:
df_fhv_2.filter(df_fhv_2.pickup_date == "2019-10-15").count()

62610

question 4

In [15]:
df_fhv_2.select(F.max(df_fhv_2.trip_duration_hours)).show()

+------------------------+
|max(trip_duration_hours)|
+------------------------+
|                631152.5|
+------------------------+



question 6

In [19]:
df_fhv_2.createOrReplaceTempView('fhv')

In [16]:
zone_df = spark.read \
        .option("InferSchema", "true") \
        .parquet('gs://airflow-lkdezoomcamp/raw/zones/taxi_zone_lookup.parquet')

In [20]:
zone_df.createOrReplaceTempView('zone')

In [21]:
df_fhv_least_pickup = spark.sql("""
SELECT 
    zone.Zone,
    COUNT(1) AS number_records
FROM fhv
LEFT JOIN zone
       ON fhv.PULocationID = zone.LocationID
GROUP BY 1
ORDER BY 2
LIMIT 1
""")

In [22]:
df_fhv_least_pickup.show()

+-----------+--------------+
|       Zone|number_records|
+-----------+--------------+
|Jamaica Bay|             1|
+-----------+--------------+

